In [1]:
import numpy as np
import pandas as pd
np.random.seed(8)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('train.csv')
test_cm = pd.read_csv('test.csv')
test_lb = pd.read_csv('test_labels.csv')
test_all = pd.merge(test_cm, test_lb, on='id')
test = test_all[test_all['toxic'] != -1]

Using TensorFlow backend.


In [2]:
train.shape

(159571, 8)

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [4]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [5]:
X_train = train["comment_text"].values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].values
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [6]:
vocab = 100000
maxlen = 200
embed_size = 300

In [7]:
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train) + list(X_test))

In [8]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [9]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))


Loaded 2000000 pre-trained words


In [10]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [17]:
def make_model(k=[2,3,4,5], activation='tanh', filters=32, Sdroprate=0.4, droprate=0.1):

    inp = Input(shape=(maxlen, ))
    x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
    x = SpatialDropout1D(Sdroprate)(x)
    x = Reshape((maxlen, embed_size, 1))(x)
    
    conv = dict()
    maxpool = dict()
    for h in k:
        conv[h] = Conv2D(filters, kernel_size=(h, embed_size), activation=activation)(x)
        maxpool[h] = MaxPool2D(pool_size=(maxlen - h + 1, 1))(conv[h])
        
    y = Concatenate(axis=1)([pool for key,pool in maxpool.items()])
    
    y = Flatten()(y)
    y = Dropout(droprate)(y)

    outp = Dense(6, activation="sigmoid")(y)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
model = make_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 200, 300)     30000000    input_5[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_5 (SpatialDro (None, 200, 300)     0           embedding_5[0][0]                
__________________________________________________________________________________________________
reshape_5 (Reshape)             (None, 200, 300, 1)  0           spatial_dropout1d_5[0][0]        
__________________________________________________________________________________________________
conv2d_14 

In [21]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits = 1, test_size = 0.1)

In [23]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
my_classifier = KerasClassifier(make_model, batch_size=256)

validator = GridSearchCV(my_classifier, param_grid={'k': [[2,3,5,7],[3,5,7]],
                                                    'activation': ['tanh'],
                                                   'epochs': [5],
                                                   'filters': [32],
                                                   'Sdroprate': [0.3],
                                                   'droprate': [0.1]},
                                                    cv = cv, 
                                                    scoring = 'roc_auc', 
                                                    n_jobs=1)

In [24]:
grid_result = validator.fit(x_train, y_train)

Epoch 1/5
143613/143613 [==============================] - 629s 4ms/step - loss: 0.0795 - acc: 0.9752
Epoch 2/5
143613/143613 [==============================] - 637s 4ms/step - loss: 0.0482 - acc: 0.9820
Epoch 3/5
143613/143613 [==============================] - 631s 4ms/step - loss: 0.0447 - acc: 0.9829
Epoch 4/5
143613/143613 [==============================] - 622s 4ms/step - loss: 0.0424 - acc: 0.9837
Epoch 5/5
143613/143613 [==============================] - 626s 4ms/step - loss: 0.0407 - acc: 0.9841
Epoch 1/5
143613/143613 [==============================] - 595s 4ms/step - loss: 0.0856 - acc: 0.9740
Epoch 2/5
143613/143613 [==============================] - 612s 4ms/step - loss: 0.0495 - acc: 0.9814
Epoch 3/5
143613/143613 [==============================] - 630s 4ms/step - loss: 0.0454 - acc: 0.9826
Epoch 4/5
143613/143613 [==============================] - 644s 4ms/step - loss: 0.0431 - acc: 0.9833
Epoch 5/5
143613/143613 [==============================] - 621s 4ms/step - loss: 0

In [25]:
print("Best score: %f with %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model

Best score: 0.988950 with {'Sdroprate': 0.3, 'activation': 'tanh', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'k': [2, 3, 5, 7]}
0.988950 (0.000000) with: {'Sdroprate': 0.3, 'activation': 'tanh', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'k': [2, 3, 5, 7]}
0.988778 (0.000000) with: {'Sdroprate': 0.3, 'activation': 'tanh', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'k': [3, 5, 7]}


In [26]:
y_pred = best_model.predict(x_test, batch_size=256)

In [27]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score for keras model: %.6f \n" % (score))


 roc_auc score for keras model: 0.982317 



In [28]:
best_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 200, 300)     30000000    input_8[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_8 (SpatialDro (None, 200, 300)     0           embedding_8[0][0]                
__________________________________________________________________________________________________
reshape_8 (Reshape)             (None, 200, 300, 1)  0           spatial_dropout1d_8[0][0]        
__________________________________________________________________________________________________
conv2d_25 

In [18]:
print(grid_result.best_estimator_.model)

In [19]:
print(grid_result.scorer_)

make_scorer(roc_auc_score, needs_threshold=True)
